In [1]:
# Initial Imports
import numpy as np
import time
import sys

In [2]:
# Instantiating base state
dicenum = 6
cards = ["BONUS: 300"]*12 + ["BONUS: 400"]*10 + ["BONUS: 500"]*8 + ["NO DICE"]*8 + ["FILL 1000"]*6 + ["MUST BUST"]*4 + ["VENGEANCE 2500"]*4 + ["DOUBLE TROUBLE"]*2
scoring_dict = {(5,): 50, (1,): 100, (2,2,2): 200, (3,3,3): 300, (4,4,4): 400, (5,5,5): 500, (6,6,6): 600, (1,1,1): 1000, (1,2,3,4,5,6): 1500}
draw_pile = cards.copy()
np.random.shuffle(draw_pile)
dice = []
saved_dice = []
turn_score = 0
players = 0
players_dict = dict()
status = -1

In [3]:
# Status codes are used for determining the state of a player's turn:
# status -1 is not rolled yet, status 0 is normal with no dice saved this throw, status 1 normal with dice saved this throw, status 2 draw again without modifying turn score, status 3 turn is over

In [4]:
# Defining function to draw a card - also shuffles if needed and resets dice
def draw():
    global dicenum
    global draw_pile
    if len(draw_pile) < 1:
        print("New Deck!")
        draw_pile = cards.copy()
        np.random.shuffle(draw_pile)
    else:
        print(f"Remaining cards: {len(draw_pile)}")
    dicenum = 6
    return draw_pile.pop()

In [5]:
# Defining function to roll dice and identify possible scoring sets
def roll_dice():
    global dice
    global status
    if status == 0:
        print("You must save some of your scoring dice before rolling again")
        print(dice)
        return dice
    roll = np.random.randint(low=1, high=7, size=dicenum)
    dice = roll.tolist()
    print(dice)
    bust = True
    dice_counts = dict()
    for num in range(1,7):
        dice_counts[num] = dice.count(num)
    if set(dice_counts.values()) == {1}:
        print("Straight! 1500 points")
        bust = False
    else:
        for key in dice_counts.keys():
            if dice_counts[key] == 6:
                print(f"Two sets of {key}: {2*scoring_dict[(key, key, key)]} points")
                bust = False
            elif dice_counts[key] > 2:
                print(f"Set of {key}: {scoring_dict[(key, key, key)]} points")
                bust = False
            if (key in [1, 5]) and (dice_counts[key] > 0):
                print(f"{dice_counts[key]} {key}s: {scoring_dict[(key,)]} points each (up to {dice_counts[key]*scoring_dict[(key,)]})")
                bust = False
    if bust:
        print("Bust!")
        status = 3
    return dice

In [6]:
# Defining function to save scoring dice
def save_dice():
    global dice
    global dicenum
    global saved_dice
    global turn_score
    global status
    if status == -1:
        print("No dice to save!")
        return dice
    while True:
        scoring_set = input("Which dice would you like to save?")
        try:
            if len(scoring_set) == 1:
                scoring_set = [int(scoring_set)]
            else:
                scoring_set = scoring_set.split(",")
                scoring_set = [int(entry.strip()) for entry in scoring_set]
            break
        except:
            print("Unable to parse - please enter single scoring set of comma-separated dice values")
            return dice
    if tuple(scoring_set) not in scoring_dict.keys():
        print("Not a valid scoring set")
        return dice
    try:
        temp_dice = list()
        for die in scoring_set:
            dice.remove(die)
            temp_dice.append(die)
            dicenum -= 1
        saved_dice.append(temp_dice)
        turn_score += scoring_dict[tuple(scoring_set)]
        status = 1
    except:
        print("Scoring set not in dice")
    print(f"Dice: {dice} \nSaved Dice: {saved_dice} \nTurn Score: {turn_score}")
    if dicenum == 0:
        print("Fill!")
        status = 2
    return dice

In [7]:
# Defining general function to get player input
def get_input(options={"0": "quit"}):
    print("What would you like to do?")
    time.sleep(2)
    while True:
        action = input(f"Choose one of the following options: {options}").lower().strip()
        if (action in options.values()) or (action in options.keys()):
            for option in options.items():
                if action in option:
                    return options_guide[option[1]]()
            break
        else:
            print("Not a valid option - try entering the number or string for one of the options provided")

In [8]:
# Defining function to end turn - kind of basic, but I want this to be in the options guide
def end_turn():
    global dice
    global status
    global players_dict
    global player
    if status < 1:
        print("You can't end your turn until you've chosen some valid scoring dice to save")
    else:
        players_dict[player] += turn_score
        status = 3
    return dice

In [9]:
# Options guide is the index for the functions
options_guide = {"quit": sys.exit, "draw": draw, "roll dice": roll_dice, "save dice": save_dice, "end turn": end_turn}

In [10]:
# Collecting the number of players from the user
while True:
    try:
        players = int(input("How many players? Enter a number: "))
    except:
        print("Not a valid number of players - please enter an integer")
        continue
    if (players > 1):
        break
    else:
        print("You can't play with fewer than two people!")
        continue

How many players? Enter a number:  2


In [11]:
for i in range(players):
    playernum = i + 1
    players_dict[f"Player {playernum}"] = 0

In [12]:
# Turn loop - this contains the actual gameplay
while True:
    for player in players_dict.keys():
        status = -1
        turn_score = 0
        saved_dice.clear()
        dice.clear()
        print(f"Current scores: {players_dict}")
        print(f"{player}'s turn")
        # Draw loop
        while True:
            card = get_input({"1": "draw", "2": "end turn", "0": "quit"})
            if status == 3:
                break
            else:
                print(card)
                if card == "NO DICE":
                    print("Sorry, no dice!")
                    status = 3
                    break
                status = -1
                # Roll Dice loop
                while status < 2:
                    dice = get_input({"1": "roll dice", "2": "save dice", "3": "end turn", "0": "quit"})
                    if status > 1:
                        break
                    elif (dice == []) or (status == 1):
                        continue
                    else:
                        status = 0
            if status == 3:
                break
            status = 0
    break #temporary one-turn limit until I figure out why quit isn't working

Current scores: {'Player 1': 0, 'Player 2': 0}
Player 1's turn
What would you like to do?


Choose one of the following options: {'1': 'draw', '2': 'end turn', '0': 'quit'} 1


Remaining cards: 54
BONUS: 300
What would you like to do?


Choose one of the following options: {'1': 'roll dice', '2': 'save dice', '3': 'end turn', '0': 'quit'} 1


[1, 3, 2, 5, 2, 4]
1 1s: 100 points each (up to 100)
1 5s: 50 points each (up to 50)
What would you like to do?


...

Choose one of the following options: {'1': 'roll dice', '2': 'save dice', '3': 'end turn', '0': 'quit'} 1


[2, 1, 6, 5, 3]
1 1s: 100 points each (up to 100)
1 5s: 50 points each (up to 50)
What would you like to do?


Choose one of the following options: {'1': 'roll dice', '2': 'save dice', '3': 'end turn', '0': 'quit'} 0


SystemExit: 

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3449: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
